# **Proyecto: 🌍✈️ Sistema integrado de gestión y recomendación de viajes**
### **Materia:** Base de Datos 2025
## Alumno: Delfina González

Este trabajo simula la integración de datos diferentes bases de datos para modelar un sistema de gestión y recomendación de viajes. El sistema **almacena** datos de usuarios, destinos, hoteles, actividades, y realiza **recomendaciones** personalizadas a partir de datos distribuidos en **Neo4j, MongoDB y Redis.**

*En el mundo actual, los viajes y el turismo generan grandes cantidades de información sobre usuarios, destinos, alojamientos y actividades. Las empresas turísticas necesitan sistemas que no solo registren esta información, sino que también permitan realizar análisis, recomendaciones personalizadas y gestionar reservas de manera eficiente.*

Funcionalidades:
 - Almacenar información de usuarios,  historial de reservas, destinos turísticos, hoteles, actividades disponibles y precios asociados. 
 - Gestionar datos temporales como búsquedas recientes o reservas en proceso. 
 - Relaciones de conocimiento entre usuarios, relaciones de usuarios y destinos. 

Estructura del proyecto:

    - PARTE 0: "Configuración y conexiones"
    - PARTE A: "Carga inicial de Datasets"
    - PARTE B: "Consultas"
    - PARTE C: "Estadísticas y Gráficos"
    - PARTE D: "Modificaciones"
    - PARTE F: "Cierre de sesiones"

# PARTE 0: Configuración y conexiones

    Se cargan las bibliotecas
    Se establecen las conexiones con Neo4j, MongoDB y Redis.

In [562]:
import os, time
import json
import random
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
from dotenv import load_dotenv
from neo4j import GraphDatabase
from pymongo import MongoClient
import redis
from datetime import datetime

# --- Configuración ---
load_dotenv()
print("Esperando servicios (5s)...")
time.sleep(5)

NEO4J_HOST = "bolt://neo4j:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "neo4j123")
MONGO_URI = f"mongodb://{os.getenv('MONGO_INITDB_ROOT_USERNAME', 'admin')}:{os.getenv('MONGO_INITDB_ROOT_PASSWORD', 'admin123')}@mongo:27017/"
REDIS_HOST = "redis"
REDIS_PORT = 6379
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "redis123")

print("Variables cargadas y hosts definidos.")

# --- Conexiones ---
def connect_neo4j():
    try:
        driver = GraphDatabase.driver(NEO4J_HOST, auth=(NEO4J_USER, NEO4J_PASSWORD))
        with driver.session() as s:
            s.run("RETURN 1")
        print("✅ Conectado a Neo4j")
        return driver
    except Exception as e:
        print(f"❌ Error al conectar a Neo4j: {e}")
        return None

def connect_mongo():
    try:
        client = MongoClient(MONGO_URI)
        client.server_info()
        print("✅ Conectado a MongoDB")
        return client
    except Exception as e:
        print(f"❌ Error al conectar a MongoDB: {e}")
        return None

def connect_redis():
    try:
        r = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, password=REDIS_PASSWORD, decode_responses=True)
        r.ping()
        print("✅ Conectado a Redis")
        return r
    except Exception as e:
        print(f"❌ Error al conectar a Redis: {e}")
        return None

neo4j_driver = connect_neo4j()
mongo_client = connect_mongo()
redis_client = connect_redis()

DB_NAME = "base_proyecto"
db = mongo_client[DB_NAME] if mongo_client else None

# Cargo constantes
DESTINO_ID_A_CIUDAD = {
    1: "Bariloche",
    2: "Cancún",
    3: "Madrid",
    4: "Roma",
    5: "Mendoza",
    6: "Ushuaia",
    7: "Puerto Iguazú",
    8: "Salta",
    9: "Río de Janeiro",
    10: "Punta Cana",
    11: "El Calafate",
    12: "Santiago",
    13: "Nueva York",
    14: "Tucumán",
    15: "Mar del Plata"
}

Esperando servicios (5s)...
Variables cargadas y hosts definidos.
✅ Conectado a Neo4j
✅ Conectado a MongoDB
✅ Conectado a Redis


# PARTE A: Carga inicial de Datasets

## 1. Carga de Datos Base y Limpieza
    Se cargan los datos de los archivos JSON requeridos y se limpian las bases de datos.

In [563]:
DATA_DIR = "data"
required_files = ["usuarios.json", "destinos.json", "hoteles.json", "actividades.json", "reservas.json"]
data = {}
missing = []

for f_name in required_files:
    f_path = os.path.join(DATA_DIR, f_name)
    if os.path.exists(f_path):
        with open(f_path, encoding="utf-8") as f:
            data[f_name.split('.')[0]] = json.load(f)
    else:
        missing.append(f_name)

if missing:
    print(f"⚠️ No se encontraron los siguientes archivos en '{DATA_DIR}/': {', '.join(missing)}")
    print("Por favor, asegúrate de tener todos los archivos JSON necesarios.")
else:
    print("✅ Archivos de datos cargados correctamente.")
    print("-" * 30)

# --- Limpieza de Bases de Datos ---
if neo4j_driver:
    with neo4j_driver.session() as s:
        s.run("MATCH (n) DETACH DELETE n")
        print("✅ Neo4j limpio.")

if mongo_client:
    mongo_client.drop_database(DB_NAME)
    print(f"✅ MongoDB ({DB_NAME}) limpio.")

if redis_client:
    for key in redis_client.scan_iter("tp_*"): 
        redis_client.delete(key)
    print("✅ Claves temporales de Redis limpias.")

✅ Archivos de datos cargados correctamente.
------------------------------
✅ Neo4j limpio.
✅ MongoDB (base_proyecto) limpio.
✅ Claves temporales de Redis limpias.


## 2. Carga en MongoDB, Redis y Neo4j
Se distribuyen los datos:

    - **MongoDB:** Usuarios, Destinos, Hoteles, Actividades y Reservas Concretadas.
    - **Redis:** Reservas Pendientes (temporal) y Usuarios Conectados (set).
    - **Neo4j:** Nodos (Usuarios, Destinos) y Relaciones Sociales y de Visita.

In [564]:
if data:
    # Carga en MongoDB (Datos permanentes)
    db.usuarios.insert_many(data["usuarios"])
    db.destinos.insert_many(data["destinos"])
    db.hoteles.insert_many(data["hoteles"])
    db.actividades.insert_many(data["actividades"])
    
    # Filtrar Reservas: Concretadas a Mongo
    reservas_concretadas = [
        r for r in data["reservas"] 
        if r["estado"] == "Confirmada"
    ]
    if reservas_concretadas:
        db.reservas.insert_many(reservas_concretadas)
    
    print("✅ Datos base y Reservas concretadas cargados en MongoDB.")

    # Carga en Redis (Datos temporales: Caché y Reservas Pendientes)
    if redis_client:
        # Reservas Temporales (Pendientes)
        reservas_pendientes = [
            r for r in data["reservas"] 
            if r["estado"] == "Pendiente"
        ]
        for r in reservas_pendientes:
            clave_pend = f"viajes_reserva:pendiente:{r['reserva_id']}"
            r_str = {k: str(v) for k, v in r.items()}
            redis_client.hset(clave_pend, mapping=r_str)
            redis_client.expire(clave_pend, random.randint(30, 120))
        print(f"✅ {len(reservas_pendientes)} Reservas Pendientes cargadas en Redis.")

        # Usuarios Conectados
        usuarios_conectados = random.sample([usu["nombre"] for usu in data["usuarios"]], k=random.randint(5, 25))
        for nombre in usuarios_conectados:
            redis_client.sadd("viajes_usuarios_conectados", nombre)
        print(f"✅ {len(usuarios_conectados)} Usuarios Conectados cargados en Redis.")

    # Carga en Neo4j (Nodos y Relaciones)
    # Nodos 
    if neo4j_driver:
        with neo4j_driver.session() as s:
            # Crear Constraints
            s.run("CREATE CONSTRAINT IF NOT EXISTS FOR (u:Usuario) REQUIRE u.usuario_id IS UNIQUE")
            s.run("CREATE CONSTRAINT IF NOT EXISTS FOR (d:Destino) REQUIRE d.destino_id IS UNIQUE")

            # Nodos
            for u in data["usuarios"]:
                s.run("MERGE (u:Usuario {usuario_id:$id, nombre:$nombre})", id=u["usuario_id"], nombre=u["nombre"])
            for d in data["destinos"]:
                s.run("MERGE (d:Destino {destino_id: $id}) SET d.ciudad = $ciudad, d.precio_base = $precio_base", 
                      id=d["destino_id"], ciudad=d["ciudad"], precio_base=d["precio_base"])

    # Relaciones VISITÓ basadas en Reservas Confirmadas Pasadas
    hoy = datetime.now().date()
    reservas_validas = [
        r for r in data.get("reservas", [])
        if r.get("estado") == "Confirmada" and datetime.strptime(r["fecha_reserva"], "%Y-%m-%d").date() <= hoy
        ]
  
    with neo4j_driver.session() as s:
        if reservas_validas not in (None, []):
            for r in reservas_validas:
                u_id = r["usuario_id"]
                d_id = r["destino_id"]
                s.run(
                        """
                        MATCH (u:Usuario {usuario_id:$u}), (d:Destino {destino_id:$d})
                        MERGE (u)-[:VISITO]->(d)
                        """,
                        u=u_id, d=d_id  
                    )
            print(f"✅ {len(reservas_validas)} relaciones VISITÓ creadas en Neo4j basadas en reservas confirmadas pasadas.")
            df_visito = pd.DataFrame(reservas_validas)
            df_visito['Nombre Usuario'] = df_visito['usuario_id'].map({u["usuario_id"]: u["nombre"] for u in data["usuarios"]})
            df_visito['Ciudad Destino'] = df_visito['destino_id'].map(DESTINO_ID_A_CIUDAD)
            display(df_visito[["reserva_id", "fecha_reserva", "estado", "Nombre Usuario", "Ciudad Destino"]].sort_values("Nombre Usuario"))
        else:
            print("⚠️ No hay reservas confirmadas pasadas para crear relaciones VISITÓ.")

        # Relaciones Sociales (AMIGO_DE, FAMILIAR_DE) - Evitando duplicados e inversas
        tipos_relacion = ["AMIGO_DE", "FAMILIAR_DE"] 
            
        # Intentamos crear el triple de relaciones que usuarios
        relaciones_creadas = []

        for _ in range(len(data["usuarios"]) * 3):
                u1 = random.choice(data["usuarios"])
                u2 = random.choice(data["usuarios"])
                u1_id = u1["usuario_id"]
                u2_id = u2["usuario_id"]
                tipo = random.choice(tipos_relacion)

                # 1. Aseguramos que los IDs sean diferentes.
                if u1_id == u2_id:
                    continue
               
                query = f"""
                MATCH (u1:Usuario {{usuario_id: $u1_id}})
                MATCH (u2:Usuario {{usuario_id: $u2_id}})

                WHERE NOT (u1)-[:AMIGO_DE|FAMILIAR_DE]-(u2)
                
                MERGE (u1)-[:{tipo}]->(u2)
                """
                # Ejecutar la query con los parámetros
                resultado_relaciones = s.run(query, u1_id=u1_id, u2_id=u2_id)

                if resultado_relaciones is not None:
                # Almacenar los datos de la relación *creada*
                    relaciones_creadas.append({
                        "Usuario 1": u1["nombre"],
                        "Usuario 2": u2["nombre"],
                        "Tipo de Relación": tipo
                    })
        # Asegurar bidireccionalidad para AMIGO_DE
        query_fix_bidireccional = """
            MATCH (a:Usuario)-[:AMIGO_DE]->(b:Usuario)
            WHERE NOT (b)-[:AMIGO_DE]->(a)
            MERGE (b)-[:AMIGO_DE]->(a)
            RETURN COUNT(*) AS relaciones_creadas
            """

        try:
            result = s.run(query_fix_bidireccional).single()
            cantidad_creadas = result["relaciones_creadas"] if result else 0
            print(f"🔄 Se aseguraron {cantidad_creadas} relaciones AMIGO_DE bidireccionales.")
        except Exception as e:
            print("⚠️ Error al asegurar bidireccionalidad:", e)

        df_relaciones = pd.DataFrame(relaciones_creadas)

        print(f"✅ {df_relaciones.shape[0]} Relaciones sociales (AMIGO_DE/FAMILIAR_DE) creadas.")

        pd.set_option('display.max_rows', 100)
        pd.set_option('display.max_colwidth', 500)

        display(df_relaciones.sort_values(["Tipo de Relación", "Usuario 1"]).reset_index(drop=True))

✅ Datos base y Reservas concretadas cargados en MongoDB.
✅ 13 Reservas Pendientes cargadas en Redis.
✅ 21 Usuarios Conectados cargados en Redis.
✅ 29 relaciones VISITÓ creadas en Neo4j basadas en reservas confirmadas pasadas.


,reserva_id,fecha_reserva,estado,Nombre Usuario,Ciudad Destino
28,50,2025-04-20,Confirmada,Agustina Benítez,Río de Janeiro
15,25,2025-03-01,Confirmada,Agustina Benítez,Tucumán
2,5,2024-06-25,Confirmada,Ana Torres,Mendoza
12,18,2024-08-25,Confirmada,Andrés Pereyra,Puerto Iguazú
26,47,2025-01-25,Confirmada,Bárbara Nuñez,Ushuaia
24,41,2025-07-15,Confirmada,Carla Gómez,Bariloche
18,31,2025-09-10,Confirmada,Delfina Giménez,Mendoza
11,17,2024-07-20,Confirmada,Diego Romero,Ushuaia
8,11,2024-01-01,Confirmada,Emilia Ríos,Bariloche
19,32,2025-10-15,Confirmada,Ezequiel Blanco,Ushuaia


🔄 Se aseguraron 62 relaciones AMIGO_DE bidireccionales.
✅ 146 Relaciones sociales (AMIGO_DE/FAMILIAR_DE) creadas.


,Usuario 1,Usuario 2,Tipo de Relación
0,Ana Torres,Sofía Morales,AMIGO_DE
1,Andrés Pereyra,Carlos Colapinto,AMIGO_DE
2,Bárbara Nuñez,Ignacio Vidal,AMIGO_DE
3,Camilo Arias,Florencia Luna,AMIGO_DE
4,Camilo Arias,Carla Gómez,AMIGO_DE
...,...,...,...
141,Sofia Olmos,Máximo Álvarez,FAMILIAR_DE
142,Sofía Morales,Sofia Olmos,FAMILIAR_DE
143,Sofía Morales,Osvaldo Quintana,FAMILIAR_DE
144,Victoria Rojas,Sebastián Maradona,FAMILIAR_DE


# PARTE B: Consultas

## 1. Funciones de Consulta (a–l)
Definición de las funciones *helper* y las consultas específicas para cada base de datos.

In [ ]:
# --- FUNCIONES DE AUXILIARES PARA LAS CONSULTAS -----------------------------------------------
def query_neo4j(query, params=None):
    """Ejecuta una consulta en Neo4j y devuelve lista de diccionarios."""
    if not neo4j_driver:
        return []

    with neo4j_driver.session() as s:
        try:
            result = s.run(query, **(params or {}))
            # Convertir cada registro en dict (para Pandas)
            return [dict(r) for r in result]
        except Exception as e:
            print("⚠️ Error en query_neo4j:", e)
            return []

def query_mongo(collection_name, query):
    """Función helper para ejecutar consultas en MongoDB."""
    
    # ✅ CORRECCIÓN: Comparamos directamente con None para evitar NotImplementedError
    if db is None: 
        return []
        
    return list(db[collection_name].find(query))

def query_redis(pattern):
    """
    Función helper para ejecutar consultas en Redis.
    Decodifica condicionalmente solo el valor (v), asumiendo que la clave (k) es un string.
    """
    if redis_client is None: 
        return []

    keys = redis_client.keys(pattern)
    results = []
    
    for key in keys:
        byte_data = redis_client.hgetall(key)
        decoded_data = {}
        for k, v in byte_data.items():
            decoded_value = v.decode('utf-8') if isinstance(v, bytes) else v
            decoded_data[k] = decoded_value
            
        results.append(decoded_data)
        
    return results

# --- PUNTO A -------------------------------------------------------------------------------

def consulta_a_usuarios_bariloche():
    """Consulta 2.a: Mostrar los usuarios que visitaron 'Bariloche'."""
    q = "MATCH (u:Usuario)-[:VISITO]->(d:Destino {ciudad:'Bariloche'}) RETURN u.nombre AS nombre, u.usuario_id AS id"
    pd_a = pd.DataFrame(query_neo4j(q)).rename(columns={0: "Usuario", 1: "ID Usuario"})
    return pd_a

# --- PUNTO B -------------------------------------------------------------------------------

def consulta_b_amigos_destinos(nombre):
    """Consulta 2.b: Amigos de Juan López que visitaron un destino que él también visitó."""
    q = f"""
    MATCH (u:Usuario)-[:AMIGO_DE]->(a:Usuario {{nombre:'{nombre}'}})-[:VISITO]->(d:Destino)
    WHERE (u)-[:VISITO]->(d)
    RETURN 
        u.nombre AS `Amigo`,
        d.ciudad AS `Destino Común`
    ORDER BY `Amigo` ASC, `Destino Común` ASC
    LIMIT 5
    """

    try:
        resultados = query_neo4j(q)
        df_b = pd.DataFrame(resultados)
    except Exception as e:
        print("⚠️ Error ejecutando consulta Neo4j:", e)
        return pd.DataFrame()

    if df_b.empty:
        print(f"⚠️ No se encontraron amigos que hayan visitado destinos comunes con {nombre}.")
    else:
        print(f"✅ Amigos de {nombre} que visitaron destinos comunes:")
        
    return df_b

# --- PUNTO C -------------------------------------------------------------------------------

def consulta_c_sugerir_destinos(nombre, preferencia_de_búsqueda):
    """
    Consulta 2.c mejorada:
    Sugerir destinos que no haya visitado el usuario ni sus amigos.
    Permite ordenar por popularidad o precio (menor o mayor).
    """

    preferencia = preferencia_de_búsqueda.strip().lower() if preferencia_de_búsqueda else "popularidad"
    
    # --- Estructura para la EXCLUSIÓN (común a todas las consultas) ---
    
    exclusion_base = f"""
    MATCH (u:Usuario {{nombre:'{nombre}'}})
    OPTIONAL MATCH (u)-[:VISITO]->(d_user:Destino)
    OPTIONAL MATCH (u)-[:AMIGO_DE]->(p:Usuario)-[:VISITO]->(d_friend:Destino)
    WITH u, collect(DISTINCT d_user) + collect(DISTINCT d_friend) AS no_sugerir_list
    """
    
    if preferencia == "mayor precio":
        q = exclusion_base + """
        MATCH (d:Destino)
        WHERE NOT d IN no_sugerir_list
        RETURN d.ciudad AS Destino_Sugerido, d.precio_base AS Precio
        ORDER BY Precio DESC, Destino_Sugerido ASC
        LIMIT 3
        """

    elif preferencia == "menor precio":
        q = exclusion_base + """
        MATCH (d:Destino)
        WHERE NOT d IN no_sugerir_list
        RETURN d.ciudad AS Destino_Sugerido, d.precio_base AS Precio
        ORDER BY Precio ASC, Destino_Sugerido ASC
        LIMIT 3
        """ 

    else:  # Popularidad
        preferencia = "popularidad"
        q = exclusion_base + """
        MATCH (d:Destino)<-[:VISITO]-(otros:Usuario)
        WITH no_sugerir_list, d, count(otros) AS popularidad
        WHERE NOT d IN no_sugerir_list
        RETURN d.ciudad AS Destino_Sugerido, popularidad AS Valoracion
        ORDER BY popularidad DESC, Destino_Sugerido ASC
        LIMIT 3
        """

    # Ejecutar la consulta con parámetro
    try:
        resultados = query_neo4j(q)
    except Exception as e:
        print("⚠️ Error ejecutando consulta Neo4j:", e)
        return pd.DataFrame()

    if not resultados:
        print(f"⚠️ No se encontraron destinos sugeridos para {nombre}.")
        return pd.DataFrame()

    df_c = pd.DataFrame(resultados)

    print(f"✅ Destinos sugeridos para {nombre} ordenados por '{preferencia}':")
    
    return df_c

# --- PUNTO D -------------------------------------------------------------------------------

def consulta_d_recomendar_por_amigos(usuario):
    """Consulta 2.d: Recomendar destinos basados en viajes de amigos (no visitados por el usuario)."""
    q = f"""MATCH (u:Usuario {{nombre:'{usuario}'}})-[:AMIGO_DE]->(a:Usuario)-[:VISITO]->(d:Destino)
           WHERE NOT (u)-[:VISITO]->(d)
           RETURN DISTINCT a.nombre AS Referente, d.ciudad AS Destino, d.destino_id AS `Destino ID` LIMIT 5"""
    
    resultado_relaciones = query_neo4j(q)

    if not resultado_relaciones:
        print("⚠️ No se encontraron viajes basados en amigos para recomendar.")
        return None
    else:
        df_d = pd.DataFrame(resultado_relaciones) 

    return df_d

# --- PUNTO E -------------------------------------------------------------------------------

def consulta_e_hoteles_destinos_recomendados(df_recomendaciones):
    """Lista los hoteles en los destinos recomendados basándose en el Destino ID."""
    
    if df_recomendaciones is None or df_recomendaciones.empty:
        print("No hay destinos recomendados.")
        return pd.DataFrame()
        
    ciudades_ids = [int(i) for i in df_recomendaciones['Destino ID'].unique().tolist()]
        
    if not ciudades_ids:
        return pd.DataFrame()

    q = {"destino_id": {"$in": ciudades_ids}}
    hoteles_data = query_mongo("hoteles", q)
    
    if not hoteles_data:
        print(f"No se encontraron hoteles en los IDs: {ciudades_ids}")
        return pd.DataFrame()

    df_e = pd.DataFrame(hoteles_data)
 
    df_e['Ciudad'] = df_e['destino_id'].map(DESTINO_ID_A_CIUDAD)
    
    df_final = df_e[[
        'nombre', 
        'Ciudad', 
        "precio", 
        'servicios'
    ]].sort_values("precio", ascending=True).rename(columns={"nombre": "Hotel"})
    
    print("\n✅ Hoteles en Destinos Recomendados:")
        
    return df_final

# --- PUNTO F -------------------------------------------------------------------------------

def consulta_f_reservas_en_proceso():
    """Consulta 2.f: Ver las reservas en proceso (Redis) usando la función helper."""
    
    reservas = query_redis("viajes_reserva:pendiente:*")

    df_f = pd.DataFrame(reservas)[["reserva_id", "usuario_id", "destino_id", "fecha_reserva", "estado","precio_total"]]
    
    df_f = df_f.sort_values(by=['usuario_id', 'fecha_reserva'], ascending=True)
    
    print(f"\n✅ {df_f.shape[0]} Reservas en Proceso (Obtenidas de Redis):")
    
    return df_f

# --- PUNTO G -------------------------------------------------------------------------------

def consulta_g_usuarios_conectados():
    """Consulta 2.g: Listar los usuarios conectados actualmente (Redis)."""
    q = query_redis("viajes_usuarios_conectados")
    return [u for u in q] 

# --- PUNTO H -------------------------------------------------------------------------------

def consulta_h_destinos_baratos(precio_max=100000):
    """Consulta 2.h: Mostrar los destinos con precio inferior a $100.000 (Mongo)."""
    q = {"precio_base": {"$lt": precio_max}}
    return pd.DataFrame(query_mongo("destinos", q))[['ciudad', 'pais', 'precio_base']]

# --- PUNTO G -------------------------------------------------------------------------------

def consulta_i_hoteles_destino(ciudad="Jujuy"):
    """Consulta 2.i: Mostrar todos los Hoteles de un destino (Mongo)."""
    q = {"ciudad": ciudad}
    return pd.DataFrame(query_mongo("hoteles", q))[['nombre', 'ciudad', 'precio', 'servicios']]

# --- PUNTO J -------------------------------------------------------------------------------

def consulta_j_cantidad_hoteles(ciudad="Bariloche"):
    """Consulta 2.j: Mostrar la cantidad de hoteles de un destino (Mongo)."""
    return db.hoteles.count_documents({"ciudad": ciudad}) if db else 0

# --- PUNTO K -------------------------------------------------------------------------------

def consulta_k_actividades_por_tipo(ciudad="Ushuaia", tipo="aventura"):
    """Consulta 2.k: Mostrar las actividades de una ciudad y tipo (Mongo)."""
    q = {"ciudad": ciudad, "tipo": tipo}
    return pd.DataFrame(query_mongo("actividades", q))[['nombre', 'ciudad', 'tipo', 'precio']]

# --- PUNTO L -------------------------------------------------------------------------------

def consulta_l_reservas_por_usuario():
    """Consulta 2.l: Mostrar la cantidad de reservas concretadas de cada usuario (Mongo)."""
    if not db: return pd.DataFrame()
    q = [{'$group': {'_id': '$usuario_id', 'cantidad_reservas': {'$sum': 1}}},
         {'$sort': {'cantidad_reservas': -1}}]
    
    resultados = list(db.reservas.aggregate(q))
    
    # Unir con nombres de usuario de la colección 'usuarios'
    usuarios_df = pd.DataFrame(list(db.usuarios.find({}, {"usuario_id": 1, "nombre": 1, "_id": 0})))
    reservas_df = pd.DataFrame(resultados).rename(columns={'_id': 'usuario_id'})
    
    df_final = pd.merge(reservas_df, usuarios_df, on='usuario_id', how='left')
    return df_final[['nombre', 'cantidad_reservas']].fillna("Desconocido")
 
# ---- Funciones auxiliares para mostrar verificaciones -------------------------------------

def obtener_amigos_y_destinos_visitados(nombre_referencia):
    """
    Obtiene los amigos de un usuario y los destinos que visitaron.
    Sin APOC y sin reduce(), el formateo se hace del lado Python.
    """

    q = f"""
    MATCH (user:Usuario {{nombre: '{nombre_referencia}'}})-[:AMIGO_DE]-(amigo:Usuario)
    OPTIONAL MATCH (amigo)-[:VISITO]->(d:Destino)
    RETURN 
        amigo.nombre AS Amigo,
        collect(DISTINCT coalesce(d.ciudad, "Ninguno")) AS DestinosVisitados
    """

    try:
        resultados = query_neo4j(q)
    except Exception as e:
        print("⚠️ Error ejecutando consulta Neo4j:", e)
        return pd.DataFrame()

    if not resultados:
        print(f"⚠️ No se encontraron amigos o destinos para {nombre_referencia}.")
        return pd.DataFrame()
    df_amigos = pd.DataFrame(resultados).rename(columns={0: "Amigo/a", 1: "Destinos Visitados"})

        
    return df_amigos 





## 2. Resultados de Consultas Integradas (a–l)



    a) Mostrar los usuarios que visitaron “Bariloche”.

In [566]:
df_a = consulta_a_usuarios_bariloche()
if df_a is not None:
    display(df_a)

,nombre,id
0,Carla Gómez,3
1,Emilia Ríos,11
2,Juan López,2


    b) Mostrar los amigos de Juan López que visitaron algún destino que visitó él, mostrando el nombre del usuario y el destino.

In [567]:
# Primero listamos sus amigos y destinos visitados, para poder verificar los resultados.
nombre_usuario = "Juan López"
df_amigos_visitados = obtener_amigos_y_destinos_visitados(nombre_usuario)

if df_amigos_visitados.empty:
    print(f"⚠️ Resultado vacío para '{nombre_usuario}'. Revisa los mensajes de debug anteriores.")
else:
    print(f"✅ Amigos de {nombre_usuario} y destinos visitados:")
    display(df_amigos_visitados)

✅ Amigos de Juan López y destinos visitados:


,Amigo,DestinosVisitados
0,Nadia Ponce,[Ninguno]
1,Pablo Olmos,[Ninguno]
2,Melina Funes,[Ninguno]


In [568]:
df_b = consulta_b_amigos_destinos("Juan López")
if df_b is not None:
    display(df_b)

⚠️ No se encontraron amigos que hayan visitado destinos comunes con Juan López.


""


    c) Sugerir destinos a un usuario que no haya visitado él ni sus amigos.

In [569]:
usuario_para_recomendacion = "Juan López"
preferencia_de_busqueda = input("Indica la preferencia de búsqueda: 'popularidad', 'mayor precio' o 'menor precio'? (default: popularidad): ")
df_c = consulta_c_sugerir_destinos(usuario_para_recomendacion, preferencia_de_busqueda)
if df_c is not None:
    display(df_c)

✅ Destinos sugeridos para Juan López ordenados por 'popularidad':


,Destino_Sugerido,Valoracion
0,Ushuaia,4
1,Punta Cana,3
2,Río de Janeiro,3


    d) Recomendar destinos basados en viajes de amigos.

In [570]:
usuario_para_recomendacion = "Juan López"
df_d = consulta_d_recomendar_por_amigos(usuario_para_recomendacion)
if df_d is not None:
    display(df_d)

⚠️ No se encontraron viajes basados en amigos para recomendar.


    e) Listar los hoteles en los destinos recomendados del punto anterior.

In [571]:
df_e = consulta_e_hoteles_destinos_recomendados(df_d)
if df_e is not None: 
    display(df_e)

No hay destinos recomendados.


""


    f) Ver las reservas en proceso, es decir, aquellas que aún no están concretadas.

In [578]:
df_f = consulta_f_reservas_en_proceso()
if df_f is not None:    
    display(df_f)


✅ 7 Reservas en Proceso (Obtenidas de Redis):


,reserva_id,usuario_id,destino_id,fecha_reserva,estado,precio_total,actividades_ids
1,4,1,4,2026-07-10,Pendiente,100000,"[21, 22]"
6,19,20,8,2026-09-05,Pendiente,70000,NaN
0,27,27,1,2026-05-10,Pendiente,90000,"[11, 12]"
4,3,3,3,2026-05-20,Pendiente,110000,NaN
3,33,33,7,2026-11-20,Pendiente,75000,NaN
5,42,42,1,2026-08-20,Pendiente,90000,"[11, 12]"
2,48,45,7,2026-02-10,Pendiente,75000,NaN


    g) Listar los usuarios conectados actualmente.

In [579]:
df_g = consulta_g_usuarios_conectados()
if df_g is not None:
    display(df_g)

ResponseError: WRONGTYPE Operation against a key holding the wrong kind of value

    h) Mostrar los destinos con precio inferior a $100.000.

In [ ]:
df_h = consulta_h_destinos_baratos(100000)
display(df_h.head())

    i) Mostrar todos los hoteles de “Jujuy”.

In [ ]:
df_i = consulta_i_hoteles_destino("Jujuy")
display(df_i.head())

    j) Mostrar la cantidad de hoteles de un destino que elija el usuario.

In [ ]:
cantidad_hoteles = consulta_j_cantidad_hoteles("Bariloche")
print(f"Cantidad de hoteles en Bariloche: {cantidad_hoteles}")

    k) Mostrar las actividades de “Ushuaia” del tipo “aventura”.

In [ ]:
df_k = consulta_k_actividades_por_tipo("Ushuaia", "aventura")
display(df_k.head())

    l) Mostrar la cantidad de reservas concretadas de cada usuario (mostrar el nombre del usuario y la cantidad).

In [ ]:
df_l = consulta_l_reservas_por_usuario()
display(df_l.head())

# PARTE D: Estadística. Gráficos y visualizaciones (m)

    1. Destino más visitado. 
    2. Hotel más barato.   
    3. Actividad más popular.

In [ ]:
# ---------- 1) Destino más visitado ----------
# Intentamos calcularlo desde la colección 'reservas' en MongoDB.
# Si no existe, usamos Neo4j contando relaciones VISITO por destino.
def calcular_destinos_mas_visitados(top_n=10):
    # Primero, intentar por MongoDB (colección 'reservas')
    destinos_agg = []
    if "reservas" in db.list_collection_names():
        pipeline = [
            {"$group": {"_id": "$destino_id", "cantidad": {"$sum": 1}}},
            {"$sort": {"cantidad": -1}}
        ]
        destinos_agg = list(db.reservas.aggregate(pipeline))
        # Mapear destino_id a ciudad
        if destinos_agg:
            # recuperar destinos
            id_map = {d["destino_id"]: d["ciudad"] for d in db.destinos.find({}, {"_id":0,"destino_id":1,"ciudad":1})}
            for doc in destinos_agg:
                doc["ciudad"] = id_map.get(doc["_id"], f"destino_{doc['_id']}")
    # Si no hay datos en reservas, fallback a Neo4j (contar VISITO)
    if not destinos_agg:
        try:
            q = """
            MATCH (u:Usuario)-[:VISITO]->(d:Destino)
            RETURN d.destino_id AS destino_id, d.ciudad AS ciudad, count(u) AS cantidad
            ORDER BY cantidad DESC
            """
            with neo4j_driver.session() as s:
                res = s.run(q)
                destinos_agg = [{"_id": r["destino_id"], "cantidad": r["cantidad"], "ciudad": r["ciudad"]} for r in res]
        except Exception:
            destinos_agg = []
    # DataFrame para graficar y devolver
    if not destinos_agg:
        return pd.DataFrame(columns=["destino_id","ciudad","cantidad"])
    df = pd.DataFrame([{"destino_id": d["_id"], "ciudad": d.get("ciudad", f"destino_{d['_id']}"), "cantidad": d["cantidad"]} for d in destinos_agg])
    df = df.sort_values("cantidad", ascending=False).head(top_n).reset_index(drop=True)
    return df

df_destinos = calcular_destinos_mas_visitados(top_n=10)

# ---------- 1.A Gráfico: barra (conteo por destino) ----------
if not df_destinos.empty:
    plt.figure(figsize=(10,4), dpi=120)
    plt.bar(df_destinos["ciudad"], df_destinos["cantidad"])
    plt.xticks(rotation=45, ha="right")
    plt.ylabel("Cantidad de reservas / visitas")
    plt.title("Reservas por destino (top)")
    plt.grid(axis="y", linestyle="--", linewidth=0.5, alpha=0.6)
    plt.tight_layout()
    plt.show()



In [ ]:

# ---------- 2) Hotel más barato ----------
# Buscamos el hotel con menor precio en la colección 'hoteles' de MongoDB (si existe)
def obtener_hotel_mas_barato(top_n=10):
    if "hoteles" not in db.list_collection_names():
        return pd.DataFrame()
    cursor = db.hoteles.find({}, {"_id":0, "hotel_id":1, "nombre":1, "ciudad":1, "precio":1}).sort("precio", 1).limit(top_n)
    docs = list(cursor)
    if not docs:
        return pd.DataFrame()
    df = pd.DataFrame(docs)
    return df

df_hoteles_baratos = obtener_hotel_mas_barato(top_n=12)

# ---------- 2.A Gráfico: barras (hoteles, destacar el más barato) ----------
if df_hoteles_baratos.empty:
    print("⚠️ No hay datos de hoteles para graficar.")
else:
    # localizar el más barato
    idx_min = df_hoteles_baratos["precio"].idxmin()
    min_name = df_hoteles_baratos.loc[idx_min, "nombre"]
    min_price = df_hoteles_baratos.loc[idx_min, "precio"]

    plt.figure(figsize=(10,6), dpi=120)
    bars = plt.bar(df_hoteles_baratos["nombre"], df_hoteles_baratos["precio"])
    # Anotar el más barato
    for i, rect in enumerate(bars):
        h = rect.get_height()
        plt.annotate(f"{int(h):,}", xy=(rect.get_x() + rect.get_width()/2, h), xytext=(0,4), textcoords="offset points", ha="center", va="bottom", fontsize=8)
    plt.xticks(rotation=45, ha="right")
    plt.ylabel("Precio del hotel ($)")
    plt.title(f"Hoteles más económicos (top) — hotel más barato: {min_name} (${int(min_price):,})")
    plt.grid(axis="y", linestyle="--", linewidth=0.5, alpha=0.6)
    plt.tight_layout()
    plt.show()


In [ ]:

# ---------- 3) Actividad más popular ----------
# Estrategia:
# - Si existe colección 'reservas_actividades' (vínculo explícito), usarla.
# - Si no existe, usar como proxy: contar visitas a la ciudad de cada actividad (Neo4j VISITO) y sumar por actividad en esa ciudad.
def calcular_actividad_mas_popular():
    coll_names = db.list_collection_names()
    candidate = None
    for name in ["reservas_actividades", "res_actividades", "reservas_actividad"]:
        if name in coll_names:
            candidate = name
            break
    if candidate:
        pipeline = [
            {"$group": {"_id": "$actividad_id", "cantidad": {"$sum": 1}}},
            {"$sort": {"cantidad": -1}}
        ]
        agg = list(db[candidate].aggregate(pipeline))
        if agg:
            id_map = {a["actividad_id"]: a["nombre"] for a in db.actividades.find({}, {"_id":0,"actividad_id":1,"nombre":1})}
            df = pd.DataFrame([{
                "actividad_id": a["_id"],
                "cantidad": a["cantidad"],
                "nombre": id_map.get(a["_id"], f"act_{a['_id']}")
            } for a in agg])
            return df

    # Fallback: usar VISITO en Neo4j como proxy de popularidad
    try:
        q = """
        MATCH (u:Usuario)-[:VISITO]->(d:Destino)
        RETURN d.ciudad AS ciudad, count(u) AS visitas
        """
        with neo4j_driver.session() as s:
            res = s.run(q)
            visitas_por_ciudad = {r["ciudad"]: r["visitas"] for r in res}
    except Exception:
        visitas_por_ciudad = {}

    activities = list(db.actividades.find({}, {"_id":0, "actividad_id":1, "nombre":1, "ciudad":1}))
    if not activities:
        return pd.DataFrame()

    rows = []
    for act in activities:
        city_visits = visitas_por_ciudad.get(act["ciudad"], 0)
        rows.append({
            "actividad_id": act["actividad_id"],
            "nombre": act["nombre"],
            "ciudad": act["ciudad"],
            "score": city_visits
        })
    df = pd.DataFrame(rows)
    df = df.sort_values("score", ascending=False).reset_index(drop=True)
    return df


df_actividades_pop = calcular_actividad_mas_popular()

# ---------- 3.A Gráfico: torta (popularidad de actividades) ----------
if df_actividades_pop.empty:
    print("⚠️ No hay datos para calcular popularidad de actividades.")
else:
    # Determinar qué columna usar (cantidad o score)
    if "cantidad" in df_actividades_pop.columns:
        valores = df_actividades_pop["cantidad"].astype(int)
        etiquetas = df_actividades_pop["nombre"]
    else:
        valores = df_actividades_pop["score"].astype(int)
        etiquetas = df_actividades_pop["nombre"]

    # Tomar top 10 actividades más populares
    top_n = min(10, len(valores))
    valores = valores[:top_n]
    etiquetas = etiquetas[:top_n]

    # Si hay muchas, agrupar las menores en "Otras"
    if len(etiquetas) > 8:
        top_k = 7
        etiquetas_top = etiquetas[:top_k].tolist()
        valores_top = valores[:top_k].tolist()
        restantes = valores[top_k:]
        etiquetas_top.append("Otras")
        valores_top.append(restantes.sum())
        etiquetas, valores = etiquetas_top, valores_top

    plt.figure(figsize=(7,7), dpi=120)
    plt.pie(
        valores,
        labels=etiquetas,
        autopct=lambda pct: f"{pct:.1f}%\n({int(round(pct/100*sum(valores)))})",
        startangle=90
    )
    plt.title("Actividades más populares (top)", fontsize=12)
    plt.axis("equal")  # mantener proporción circular
    plt.tight_layout()
    plt.show()




In [ ]:
# ---------- Resumen textual de las 3 métricas ----------
print("\n" + "="*60)
if not df_destinos.empty:
    top = df_destinos.iloc[0]
    print(f"Destino más visitado: {top['ciudad']} — {int(top['cantidad'])} visitas/reservas (top).")
else:
    print("Destino más visitado: no disponible.")

if not df_hoteles_baratos.empty:
    cheapest = df_hoteles_baratos.iloc[0]
    print(f"Hotel más barato: {cheapest['nombre']} ({cheapest['ciudad']}) — ${int(cheapest['precio']):,}")
else:
    print("Hotel más barato: no disponible.")

if not df_actividades_pop.empty:
    top_act = df_actividades_pop.iloc[0]
    # si existe 'cantidad' la mostramos, si no mostramos 'score'
    if "cantidad" in df_actividades_pop.columns:
        cnt = int(top_act["cantidad"])
    else:
        cnt = int(top_act["score"])
    print(f"Actividad más popular: {top_act['nombre']} ({top_act.get('ciudad','-')}) — {cnt} puntos/veces.")
else:
    print("Actividad más popular: no disponible.")
print("="*60)

# PARTE D: Realizar modificación en los datos

    - Incrementar el precio de las actividades de Tucuman en 5% 
    - Agregar al hotel id=1 el servicio de SPA 
    - Eliminar el destino que desee 
    - Eliminar un usuario que desee 
    - Eliminar las relaciones AMIGO_DE para un usuario que quiera. 

In [ ]:
def modificar_precio_actividades(ciudad, pct):
    """Incrementar precio de actividades en una ciudad (MongoDB)"""
    res = db.actividades.update_many({"ciudad": ciudad}, {"$mul": {"precio": 1 + pct}})
    print(f"Incrementadas {res.modified_count} actividades en {ciudad} en un {pct*100}%. (Mongo)")

def agregar_servicio_hotel(hotel_id, servicio):
    """Agregar un servicio a un hotel (MongoDB)"""
    res = db.hoteles.update_one({"hotel_id": hotel_id}, {"$addToSet": {"servicios": servicio}})
    print(f"Servicio '{servicio}' agregado al hotel {hotel_id}. Modificados: {res.modified_count} (Mongo)")

def eliminar_usuario(nombre):
    """Eliminar un usuario de MongoDB y sus nodos/relaciones en Neo4j (Integrado)"""
    
    # 1. Eliminar de Neo4j
    q_neo4j = "MATCH (u:Usuario {nombre:$nombre}) DETACH DELETE u"
    with neo4j_driver.session() as s:
        s.run(q_neo4j, nombre=nombre)
    print(f"Usuario {nombre} eliminado de Neo4j (Nodos y Relaciones).")
    
    # 2. Eliminar de MongoDB
    db.usuarios.delete_one({"nombre": nombre})
    print(f"Usuario {nombre} eliminado de MongoDB (Colección Usuarios).")
    

# Ejecución de Modificaciones
print("\n--- Modificaciones ---\n")
modificar_precio_actividades("Bariloche", 0.1)
agregar_servicio_hotel(hotel_id=1, servicio="Spa")
eliminar_usuario("Juan López")

## PARTE F. Cierre de Conexiones
    Se cierran las conexiones a las bases de datos.

In [ ]:
if neo4j_driver:
    neo4j_driver.close()
if mongo_client:
    mongo_client.close()
if redis_client:
    redis_client.close()
print("🔒 Conexiones cerradas correctamente.")